### This is the main notebook. Use this notebook as a template to create branches. 

In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime as dt
import json

In [2]:
# Get latitiude and longitude from: https://www.matthewproctor.com/australian_postcodes
data = (r'C:\Users\TribThapa\Desktop\Thapa\ResearchFellow\PropertyAnalysis\Data\australian_postcodes.csv')

lat_long_df = pd.read_csv(data)

lat_long_df =lat_long_df[["postcode", "state", "long", "lat"]]

qld_postcodes = lat_long_df.loc[lat_long_df["postcode"] == 4000]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\TribThapa\\Desktop\\Thapa\\ResearchFellow\\PropertyAnalysis\\Data\\australian_postcodes.csv'

In [3]:
#Domain
#api_key = 'key_b16926690a50662e60bca261f8485d2a'
client_id = 'client_d68a201ec49ec7e83f3b3f8347f4af6f'
client_secret = 'secret_042060b42af3a9f3c2e4a5604961a21e'

In [5]:
response = requests.post('https://auth.domain.com.au/v1/connect/token', 
                         data={'client_id': client_id,
                               'client_secret': client_secret, 
                               'grant_type': 'client_credentials',
                               'scope':'api_listings_read',
                               'Content-Type':'text/json'})
token = response.json()
access_token=token["access_token"]
auth = {"Authorization":"Bearer "+access_token}

In [6]:
postcode = [4000]

property_data = []


# Get property data from Domain
for post in postcode:
    url = "https://api.domain.com.au/v1/listings/residential/_search"
    
    post_fields ={"listingType":"Sale",
                  "propertyTypes":["ApartmentUnitFlat","House","NewApartments","Townhouse"],
                  "locations":[{"state":"QLD",
                                "postCode":post,
                                "includeSurroundingSuburbs":False}],
                  "excludePriceWithheld": True,
                  "sort": {"sortKey": "SoldDate",
                           "direction": "Ascending",
                           "proximityTo": {"lat": -27.468391,
                                           "lon": 153.022126}},
                  "pageSize": 200,
                  "pageNumber":0,
                  "updatedSince": "2021-01-01T00:00:00.000Z"}
    
    request = requests.post(url,headers=auth,json=post_fields)
    request_json = request.json()
    #print(json.dumps(request_json, indent=4))
    property_data.append(request_json)

In [7]:
from itertools import chain
property_data_new = list(chain.from_iterable(property_data))
print(len(property_data_new))

res = len([ele for ele in property_data_new if isinstance(ele, dict)])
#res

33


In [47]:
# Extract property data    
property_type = []
property_id = []
state = []
property_price = []
features = []
bathrooms = []
bedrooms = []
carspaces = []
area = []
suburb = []
postcode = []
lat = []
long = []

for details in property_data:
    
    for num_elements in range(len(property_data_new)):
        
        #print(details[num_elements]['listing'])  
        try:            
            property_type.append(details[num_elements]["listing"]["listingType"])
        except:
            property_type.append("na")
            
        try:
            property_id.append(details[num_elements]['listing']['id'])
        except:
             property_id.append("na")
                
        try:
            state.append(details[num_elements]['listing']['propertyDetails']['state'])
        except:
            state.append("na")

        try:
            property_price.append(details[num_elements]['listing']['priceDetails']['price'])
        except:
            property_price.append("na")
            
        try: 
            features.append(details[num_elements]['listing']['propertyDetails']['features'])
        except:
            features.append("na")
            
        try: 
            bathrooms.append(details[num_elements]['listing']['propertyDetails']['bathrooms'])
        except:
            bathrooms.append("na")
            
        try: 
            bedrooms.append(details[num_elements]['listing']['propertyDetails']['bedrooms'])
        except:
            bedrooms.append("na")
            
        try: 
            carspaces.append(details[num_elements]['listing']['propertyDetails']['carspaces'])
        except:
            carspaces.append("na")
       
        try: 
            area.append(details[num_elements]['listing']['propertyDetails']['area'])
        except:
            area.append("na")            

        try: 
            region.append(details[num_elements]['listing']['propertyDetails']['region'])
        except:
            region.append("na")

        try: 
            suburb.append(details[num_elements]['listing']['propertyDetails']['suburb'])
        except:
            suburb.append("na")
            
        try: 
            postcode.append(details[num_elements]['listing']['propertyDetails']['postcode'])
        except:
            postcode.append("na")
            
        try: 
            lat.append(details[num_elements]['listing']['propertyDetails']['latitude'])
        except:
            lat.append("na")
            
        try: 
            long.append(details[num_elements]['listing']['propertyDetails']['longitude'])
        except:
            long.append("na")
            


In [48]:
#property_data

In [50]:
# Concat lists to dataframe
final_df = pd.DataFrame({"PropertyID": property_id,
                         "Type": property_type,
                         "State": state,
                         "Price": property_price,
                         "Features": features,
                         "Bathrooms": bathrooms,
                         "Bedrooms": bedrooms,
                         "Carspaces": carspaces,
                         "Area": area,
                         "Suburb": suburb,
                         "Postcode": postcode,
                         "Latitude": lat,
                         "Longitude": long})

final_df.head()

,PropertyID,Type,State,Price,Features,Bathrooms,Bedrooms,Carspaces,Area,Suburb,Postcode,Latitude,Longitude
0,2016723553,Sale,QLD,349000,"[SwimmingPool, Furnished, Gym]",1,1,1,City & North,BRISBANE CITY,4000,-27.4622,153.031
1,2016727278,Sale,QLD,140000,"[CableOrSatellite, SwimmingPool, Furnished, St...",1,na,na,City & North,BRISBANE CITY,4000,-27.4732,153.027
2,2016806859,Sale,QLD,559000,"[AirConditioning, BuiltInWardrobes, CableOrSat...",2,2,1,City & North,BRISBANE CITY,4000,-27.4705,153.03
3,2016834181,Sale,QLD,329000,"[AirConditioning, BuiltInWardrobes, SecurePark...",1,1,1,City & North,BRISBANE CITY,4000,-27.4622,153.031
4,2016837531,Sale,QLD,384000,"[AirConditioning, BuiltInWardrobes, Gas, Inter...",1,1,1,City & North,BRISBANE CITY,4000,-27.4721,153.029


In [52]:
final_df.to_csv("../Data/PropertyTraits.csv")